In [2]:
import os
import lmdb
import cv2
import glob


In [ ]:


def generate_labels_file(dataset_dir, output_file):
    with open(output_file, 'w') as f:
        for root, dirs, files in os.walk(dataset_dir):
            print(f"Inspecting directory: {root}")  # Debugging: print the directory
            print(f"Subdirectories found: {dirs}")  # Debugging: print subdirectories
            print(f"Files found: {files}")  # Debugging: print files in the directory

            for dir_name in dirs:
                folder_path = os.path.join(root, dir_name)
                print(f"Processing folder: {folder_path}")  # Debugging: folder being processed
                
                # Folder name is the label
                label = dir_name

                # Iterate through all images in the folder
                for file_name in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file_name)

                    # Check if the file is an image
                    if file_name.lower().endswith(('.jpg', '.png', '.jpeg')):
                        relative_path = os.path.join(dir_name, file_name)
                        print(f"Adding to labels: {relative_path} -> Label: {label}")  # Debugging: print what is added
                        f.write(f"{relative_path} {label}\n")
                    else:
                        print(f"Ignored file (not an image): {file_name}")  # Debugging: non-image file

    print(f"Labels file created: {output_file}")

# Paths
dataset_dir = "chars74k/train"  # Adjust path to point to your dataset
output_file = "labels.txt"

generate_labels_file(dataset_dir, output_file)


Inspecting directory: chars74k/train
Subdirectories found: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
Files found: []
Processing folder: chars74k/train\0
Adding to labels: 0\img001-00001.png -> Label: 0
Adding to labels: 0\img001-00002.png -> Label: 0
Adding to labels: 0\img001-00003.png -> Label: 0
Adding to labels: 0\img001-00004.png -> Label: 0
Adding to labels: 0\img001-00005.png -> Label: 0
Adding to labels: 0\img001-00006.png -> Label: 0
Adding to labels: 0\img001-00007.png -> Label: 0
Adding to labels: 0\img001-00008.png -> Label: 0
Adding to labels: 0\img001-00009.png -> Label: 0
Adding to labels: 0\img001-00010.png -> Label: 0
Adding to labels: 0\img001-00011.png -> Label: 0
Adding to labels: 0\img001-00012.png -> Label: 0
Adding to labels: 0\img001-00013.png -> Label: 0
Adding to labels: 0\img001-00014.png -> Label: 0
Adding to labels: 0\im

In [44]:
def write_cache(env, cache):
    with env.begin(write=True) as txn:
        for k, v in cache.items():
            txn.put(k.encode(), v)

In [47]:
def create_lmdb_dataset(output_path, image_path, label_path):
    os.makedirs(output_path, exist_ok=True)
    env = lmdb.open(output_path, map_size=1073741824)
    cache = {}
    count = 0

    with open(label_path, "r") as f:
        lines = f.readlines()
    
    for line in lines:
        img_path, label = line.strip().split("\t")
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Image not found: {img_path}")
            continue
        
        _, img_encoded = cv2.imencode('.jpg', img)
        cache[f'image-{count}'] = img_encoded.tobytes()
        cache[f'label-{count}'] = label.encode()
        count += 1

        if count % 1000 == 0:
            write_cache(env, cache)
            cache = {}

    cache['num-samples'] = str(count).encode()
    write_cache(env, cache)
    print(f"Created dataset with {count} samples.")


In [48]:
dataset_dir = "D:/anpr2/chars74k/train"
label_file = "labels.txt"  # A tab-separated file with "image_path\tlabel"
lmdb_output = "lmdb_dataset"

In [49]:
with open("labels.txt", "w") as f:
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(".jpg"):
                char_label = os.path.basename(root)
                img_path = os.path.join(root, file)
                f.write(f"{img_path}\t{char_label}\n")

In [50]:
create_lmdb_dataset(lmdb_output, dataset_dir, "labels.txt")

Created dataset with 0 samples.


In [52]:
import lmdb
import cv2
import numpy as np

# Define the path to the LMDB dataset
lmdb_path = "D:/anpr2/lmdb_dataset/data.mdb"

# Open the LMDB environment
env = lmdb.open(lmdb_path, readonly=True)

with env.begin() as txn:
    # Get the number of items in the database
    num_items = txn.stat()["entries"]
    print(f"Number of entries in LMDB: {num_items}")
    
    # Iterate through all items in the database
    for i in range(num_items):
        # Get the image and label (here it's assumed that key = index and value = image data)
        key = str(i).encode("utf-8")
        value = txn.get(key)
        
        # Deserialize the image and label (assuming it's stored as a dictionary or byte data)
        example = np.frombuffer(value, dtype=np.uint8)
        
        # Convert to image (assuming it's stored as a compressed image, like JPG/PNG)
        img = cv2.imdecode(example, cv2.IMREAD_COLOR)
        
        # Optionally, preprocess the image here (resize, grayscale, etc.)
        img_resized = cv2.resize(img, (28, 28))  # Resize for OCR model input
        
        # Extract the label (adjust based on how it's stored)
        label = get_label_for_image(i)  # You need to define how to get the correct label from the key
        print(f"Image {i}, Label: {label}")

env.close()


Error: D:/anpr2/lmdb_dataset/data.mdb: The system cannot find the path specified.
